In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping


2023-11-05 12:35:16.593415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 12:35:16.619687: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:35:16.899399: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:35:16.901908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 12:35:19.217346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
dataset = '../data/Sarcasm Detection.csv'

In [3]:
df = pd.read_csv(dataset)
df.head()

,Tweet,Label
0,I love finding out who your true friends are. ...,0
1,USER :/ bye,0
2,USER I absolutely love Kanye's voice USER,0
3,We're hiring a new Kiosk Supervisor! Think you...,0
4,Every time I see Dave Grohl with his long hipp...,0


In [4]:
df.drop_duplicates(inplace=True)

In [5]:
def cleanText(text):
    text = re.sub(r'RT @\w+: ', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

df['Tweet'] = df['Tweet'].apply(cleanText)
df.head()

,Tweet,Label
0,i love finding out who your true friends are ...,0
1,user bye,0
2,user i absolutely love kanye s voice user,0
3,we re hiring a new kiosk supervisor think you...,0
4,every time i see dave grohl with his long hipp...,0


In [6]:
cv = CountVectorizer()
X = cv.fit_transform(df['Tweet'])
y = df['Label']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test.toarray(), y_test))

model.save('../models/sarcasmDetection.h5')
pickle.dump(cv, open('../models/cvsarcasm.pkl', 'wb'))

Epoch 1/10
114/114 [==============================] - 2s 10ms/step - loss: 0.4808 - accuracy: 0.8067 - val_loss: 0.3112 - val_accuracy: 0.9055
Epoch 2/10
114/114 [==============================] - 1s 7ms/step - loss: 0.2018 - accuracy: 0.9431 - val_loss: 0.2291 - val_accuracy: 0.9143
Epoch 3/10
114/114 [==============================] - 1s 7ms/step - loss: 0.1003 - accuracy: 0.9686 - val_loss: 0.2291 - val_accuracy: 0.9110
Epoch 4/10
114/114 [==============================] - 1s 7ms/step - loss: 0.0558 - accuracy: 0.9838 - val_loss: 0.2521 - val_accuracy: 0.9099
Epoch 5/10
114/114 [==============================] - 1s 9ms/step - loss: 0.0348 - accuracy: 0.9915 - val_loss: 0.2734 - val_accuracy: 0.9077
Epoch 6/10
114/114 [==============================] - 1s 7ms/step - loss: 0.0214 - accuracy: 0.9950 - val_loss: 0.3023 - val_accuracy: 0.9099
Epoch 7/10
114/114 [==============================] - 1s 8ms/step - loss: 0.0133 - accuracy: 0.9975 - val_loss: 0.3258 - val_accuracy: 0.9077
Epoch